We already have seen decorators like `@staticmethod` to define a method as static or `@pytest.fixture` to define values in Pytest.

Decorators are wrappers which get a function and return a modified version of the function however see them as tricks to make coding easier :-). There are plenty of them since anyone can make new ones (if you are curious, see [here](https://realpython.com/blog/python/primer-on-python-decorators/)). Following are the main core decorators.


## Properties, getters and setters

An object variable can be changed directly :

In [1]:
class Dog:
    def __init__(self,name):
        self.name = name

max = Dog("Maxime")
print(max.name)
max.name = "Bobby"
print(max.name)

Maxime
Bobby


Sometime you want to check that the new value for an object variable is ok. For example you may want to check that a length is positive or that a new Dog has not the same name than another one. To do so you should use methods :

In [2]:
class Dog:
    _all_names = []  # class variable, shared by all objects of this class
    
    def __init__(self, name):
        self.name(name)  # call the method name(self,name)
        
    def name(self, name):
        if name in self._all_names:
            raise Exception("Name already used")
        else:
            self.name = name
            self._all_names.append(name)  # _all_names is a class variable 
                                          # but it can also be called from the object            
max = Dog("Maxime")
bob = Dog("Bobby")
print(Dog._all_names)
bob.name = "Maxime"   # now we have 2 Maxime...

['Maxime', 'Bobby']


This is fine except if you want to change the name after the creation of the object since:

* you still can do `bob.name = "Maxime"` and now we have 2 Maxime !
* you should do `bob.name("Maxime")` which will check and is fine but not the natural way

Therefore **we want to do `bob.name = "Maxime"` with the checking**. The decorators @property and @setter are done for that:

In [3]:
class Dog:
    _all_names = []
    
    def __init__(self, name):
        self.name = name       # call name(name)
    
    @property       # this is a decorator, it starts with @ and is just before a function
    def name(self):
        return self._name
    
    @name.setter
    def name(self, name):
        if name in self._all_names:
            raise Exception("Name already used")
        else:
            self._name = name    # the variable is now private : _name
            self._all_names.append(name)
            
max = Dog("Maxime")
bob = Dog("Bobby")
print(bob.name)       # uses bob.name()
bob.name = "Maxime"   # call bob.name("Maxime") which check if Maxime is used

Bobby


Exception: Name already used

### Decorator deleter

A third decorator can be linked to a variable which is how to delete it.

In our case when an object is deleted we need to remove its name from `_all_names` so another dog can have the same name later.

In [4]:
class Dog():
    _all_names = []
    
    def __init__(self, name):
        self.name = name        # call name(name)
    
    def __del__(self):
        del self.name           # call deleter of name
    
    @property
    def name(self):
        return self._name
    
    @name.setter
    def name(self, name):
        if name in Dog._all_names:
            raise Exception("Name already used")
        else:
            self._name = name   # here is where we store the name in the private variable _name
            Dog._all_names.append(name)

    @name.deleter
    def name(self):
        print('name deleter called')
        Dog._all_names.remove(self._name)

max = Dog("Maxime")
del max
print(Dog._all_names)

name deleter called
[]


There is another way to define properties. You define 3 fonctions, the getter, the setter and the deleter, and give them to the builtin function `property()`:

In [5]:
class Dog:
    _all_names = []
    
    def __init__(self, name):
        self.name = name

    def __del__(self):
        del self.name 
        
    def _get_name(self):
        return self._name
    
    def _set_name(self, name):
        if name in self._all_names:
            raise Exception("Name already used")
        else:
            self._name = name
            self._all_names.append(name)
            
    def _del_name(self):
        Dog._all_names.remove(self.name)
        
    name = property(_get_name, _set_name, _del_name)  # it looks like a class variable 
                                                      # but it is must be seen as a object variable
    
max = Dog("Maxime")
max.name = "Toto"
print(max.name)
print(Dog._all_names)  # oops, there is still a dog Maxime

Toto
['Maxime', 'Toto']


__Exercice__ : modify the class Dog to correct the bug.

# @classmethod

A method is a function of an object, a static method is a function of a class, a class method is also a function of a class used to create objects like `__init__()` does. It is usefull when you want different ways to build the same kind of objects.

In [6]:
class Date:
    
    def __init__(self, day, month, year):
        self.day = day
        self.month = month
        self.year = year
        
    @classmethod
    def from_string(cls, date_as_string):  # the first parameter of a class method
                                           # is the class itself
        """
        Convert string 'dd-mm-yyyy' to Date object.
        """
        day, month, year = map(int, date_as_string.split('-'))
        return cls(day, month, year)
    
d = Date.from_string("12-03-2014")
print(d.year)

2014


We have to do that because Python does not have overloading like C++ does. With overloading you can define many methods with the same name as long their arguments have different signature (meaning number of argument and their type). Since Python do not specify type of argument it makes difficult (useless) to have overloading. Therefore there is only one method `__init__`.

By the way, for those who want to do overloading, there is a module which allow to do overloading with a decorator `@overload` : https://pypi.python.org/pypi/overload/ (pip install overload)

# More

You can imagine many more decorators

* to time a function
* to log
* to memoize (i.e. cache last value returned for each args so you have the result immediatly)
* check hardware (net, IO) before running function

This [PythonDecoratorLibrary](https://wiki.python.org/moin/PythonDecoratorLibrary) has more.

{{ PreviousNext("10 Magic methods.ipynb", "../lesson4 Numpy/np01 Numpy Introduction.ipynb")}}